In [1]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import List
from pydantic import BaseModel, Field
import uuid


In [2]:

# ---------- STATE ----------
class Intent(BaseModel):
    uid: str = Field(default_factory=lambda: str(uuid.uuid4()))
    name: str
    completed: bool = False

class MyState(BaseModel):
    messages: List[str] = Field(default_factory=list)
    incompleted_intents: List[Intent] = Field(default_factory=list)
    output: str = None

In [6]:


# ---------- NODES ----------
def normal_handler(state: MyState):
    print("➡ Normal user input handling...")
    # pretend we detect an intent
    state.incompleted_intents.append(Intent(name="Fetch Loan Schemes"))
    state.output = "Detected an intent and added to incompleted list."
    return state

def intent_completion_handler(state: MyState):
    print("➡ Completing incompleted intents...")
    if state.incompleted_intents:
        # Simulate marking first incomplete intent as complete
        state.incompleted_intents[0].completed = True
        # Remove if completed
        state.incompleted_intents = [
            i for i in state.incompleted_intents if not i.completed
        ]
        state.output = "Completed an intent."
    return state

def supervisor_node(state: MyState):
    print("🕵 Supervisor checking state...")
    if state.incompleted_intents:
        print(f"Found {len(state.incompleted_intents)} incompleted intents.")
        return {'next':"intent_completion_handler"}
    else:
        print("No incompleted intents. Going to normal handler.")
        return {"next": "normal_handler"}


In [7]:


# ---------- GRAPH ----------
workflow = StateGraph(MyState)

workflow.add_node("normal_handler", normal_handler)
workflow.add_node("intent_completion_handler", intent_completion_handler)
workflow.add_node("supervisor", supervisor_node)

workflow.add_edge(START, "supervisor")
workflow.add_conditional_edges(
    "supervisor",
    lambda state: supervisor_node(state),  # This picks next node name
    {
        "normal_handler": "supervisor",
        "intent_completion_handler": "supervisor"
    }
)

# Stop when no incompleted intents AND last output came from normal handler
def end_check(state: MyState):
    if not state.incompleted_intents and "normal" in (state.output or "").lower():
        return END
    return "supervisor"

workflow.add_conditional_edges(
    "supervisor",
    end_check,
    {END: END, "supervisor": "supervisor"}
)

app = workflow.compile()

In [8]:


# ---------- RUN DEMO ----------
state = MyState(messages=["Hi, show me open cases"])
app.invoke(state)


🕵 Supervisor checking state...
No incompleted intents. Going to normal handler.
🕵 Supervisor checking state...
No incompleted intents. Going to normal handler.


TypeError: unhashable type: 'dict'